In [14]:
import numpy as np
import open3d as o3d
import os
from makegripper_points import  plot_gripper_pro_max
from scipy.spatial.transform import Rotation as R

# Include your original plot_gripper_pro_max function here or import it if it's from another module

# Load saved data
data_dir =  './pavanoutput6'

cloud_path = os.path.join(data_dir, "cloud.ply")
grasps_path = os.path.join(data_dir, "grasps.npy")

cloud = o3d.io.read_point_cloud(cloud_path)
grasp_data = np.load(grasps_path, allow_pickle=True).item()

translations = grasp_data['translations']
rotations = grasp_data['rotations']
widths = grasp_data['widths']
heights = grasp_data['heights']
scores = grasp_data['scores']
gripper_points = grasp_data['gripper_points']



def visualize_selected_grasps(grasp_indices):

    pcds = []
    dots = []
    selected_index=[]

    for idx in grasp_indices:
        if idx >= len(translations):
            print(f"Index {idx} out of range.")
            continue

        t = translations[idx]
        R_mat = rotations[idx]
        width = widths[idx]
        height = heights[idx]
        score = scores[idx]
        gripper_point = gripper_points[idx]

        # Gripper mesh
        gripper_pcd, _ = plot_gripper_pro_max(t, R_mat, width=width, depth=height, score=score, color=[0, 1, 0])
        # pcds.append(gripper_pcd)




###
        R_mat_XG = R_mat.copy() #xarm gripper
        R_mat_XG[:, [0, 2]] = R_mat_XG[:, [2, 0]]  # Swap X and Z axes in real -life gripper
        R_mat_XG[:, 1] *= -1  


        #pregrasp  
        z_axis = R_mat_XG[:, 2]  # Local z-axis (gripper heading)
        t_moved_pregrasp = t - 0.26 * z_axis  # Move backwards by 0.26



        #Grasp: Move gripper backwards along its local z-axis by 0.15 meters
        z_axis = R_mat_XG[:, 2]  # Local z-axis (gripper heading)
        t_moved = t - 0.15 * z_axis  # Move backwards by 0.15

        #xarm fingers frame
        xarm_gripT = np.eye(4)  
        xarm_gripT[:3, :3] = R_mat_XG
        xarm_gripT[:3, 3] = t_moved
        axis_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05)
        axis_frame.transform(xarm_gripT)

        approach_vector = xarm_gripT[:3, 2]  # z-axis in world frame
        # World z-axis
        world_z = np.array([0, 0, -1]) 
        cos_theta = np.dot(approach_vector, world_z) / (np.linalg.norm(approach_vector))
        angle_deg = np.rad2deg(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
        print(idx , "   ",angle_deg)
        # Filter: keep only top-down grasps
        if angle_deg >45 or t_moved[2]<0.15:
            continue  # Skip if not top-down
        

        selected_index.append(idx)
        pcds.append(axis_frame)
        pcds.append(gripper_pcd)
        # Red dots
        for p in gripper_point:
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
            sphere.translate(p)
            sphere.paint_uniform_color([1, 0, 0])
            dots.append(sphere)


###

        rpy = R.from_matrix(R_mat_XG).as_euler('xyz', degrees=True) 

        # gaxis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
        # gaxis.rotate(R_mat, center=(0, 0, 0))
        # gaxis.translate(t)
        # pcds.append(gaxis)

        print(f"\nGrasp Index: {idx}")
        print(f"  Score: {score:.4f}")
        print(f"  Rotation (roll, pitch, yaw) [deg]: {rpy}\n")

        print(f"  Pregrasp Translation (x, y, z): {t_moved_pregrasp*1000}")
        print(f"\n  Translation (x, y, z): {t_moved*1000}\n")

        print(f"  Width: {width:.4f}  | Depth: {height:.4f}")


    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
    o3d.visualization.draw_geometries([cloud, axis] + pcds + dots)
    print("selected grasp index ",selected_index)

import random
total_grasps= len(translations)
print(total_grasps)
grasps_list=list(range(len(translations)))
random.shuffle(grasps_list) 
print(grasps_list)
visualize_selected_grasps( grasp_indices=grasps_list)# ([4])

#4,


40
[6, 32, 10, 4, 37, 39, 1, 20, 31, 33, 16, 21, 25, 2, 34, 19, 27, 11, 14, 29, 22, 13, 8, 28, 35, 38, 12, 5, 15, 18, 3, 36, 7, 0, 9, 26, 23, 24, 17, 30]
6     11.20692863588825

Grasp Index: 6
  Score: 0.0788
  Rotation (roll, pitch, yaw) [deg]: [174.92333799 -10.00426678   0.90790945]

  Pregrasp Translation (x, y, z): [466.69119972 428.90296305 295.7035991 ]

  Translation (x, y, z): [485.87744744 419.47204843 187.80111634]

  Width: 0.0318  | Depth: 0.0300
32     71.85990341535795
10     83.52363765049195
4     11.749269561338494

Grasp Index: 4
  Score: 0.0971
  Rotation (roll, pitch, yaw) [deg]: [179.90210786 -11.74886747 -86.30649618]

  Pregrasp Translation (x, y, z): [546.81373417 422.61539884 290.96008309]

  Translation (x, y, z): [548.06907281 400.25138362 183.26479446]

  Width: 0.0331  | Depth: 0.0300
37     22.108375204714548
39     40.18506717148039
1     21.31267519761111

Grasp Index: 1
  Score: 0.1820
  Rotation (roll, pitch, yaw) [deg]: [-178.10073822  -21.23180538 